In [16]:
import numpy as np
import cv2

#Parameters for the first video.
pedestrian_y_threshold_1 = 0.7
pedestrian_y_threshold_2 = 0.75
y_threshold_1 = 0.72
y_threshold_2 = 0.84
parking_x_threshold = 0.15
"""
==================     STEP1   ===================
Start of: Reading input video
"""
#NOTE:
# Defining 'VideoCapture' object and reading video from a file make sure that the path and file name is correct
video = cv2.VideoCapture( 'mcgill_drive.mp4')

# Preparing variable for writer that we will use to write processed frames
writer = None

# Preparing variables for spatial dimensions of the frames
h, w = None, None

"""
End of:
Reading input video
"""
'''
Helper Methods. 
Euclid_2D calculates the euclidean distance between two pixels in the image (useful for object matching between different frames).
is_above_line_2D checks if a pixel (target) is above a line described by two other pixels (line_pt1,line_pt2) in a frame.
'''
def euclid_2D(pt1,pt2):
    return np.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)

def is_above_line_2D(line_pt1, line_pt2, target):
    a = (line_pt2[1]-line_pt1[1])/(line_pt2[0]-line_pt1[0])
    b = line_pt1[1] - line_pt1[0] * a
    return (target[1] > (a * target[0] + b))

def draw_label(img, text, pos, bg_color):
    font_scale = 0.5
    font = cv2.FONT_HERSHEY_SIMPLEX
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
    box_coords = ((pos[0], pos[1]), (pos[0] + text_width + 2, pos[1] - text_height - 2))
    cv2.rectangle(img, box_coords[0], box_coords[1], bg_color, cv2.FILLED)
    cv2.putText(img, text, (pos[0], pos[1]), font, fontScale=font_scale, color=(0,0,0), thickness=1)


"""
==================     STEP2  ===================
Start of: Loading YOLO v3 network
"""

# Loading COCO class labels from file and Opening file
with open('yolo_data/coco.names') as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]


# Loading trained YOLO v3 Objects Detector with the help of 'dnn' library from OpenCV
network = cv2.dnn.readNetFromDarknet('yolo_data/yolov3.cfg',
                                     'yolo_data/yolov3.weights')

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# Getting only output layers' names that we need from YOLO v3 algorithm
# with function that returns indexes of layers with unconnected outputs
print(network.getUnconnectedOutLayers())
layers_names_output = [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]

# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.7

# Setting threshold for filtering weak bounding boxes
# with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object
# with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

"""
==================     STEP3  ===================
Start of: Reading frames in the loop
"""

# Defining variable for counting frames at the end we will show total amount of processed frames
f = 0

# Defining variable for counting total time At the end we will show time spent for processing all frames
t = 0

# Defining loop for catching frames
total_cars_count = 0
parked_cars_count = 0
total_pedestrian_count = 0

prev_results = []

prev_boxes = []

prev_labels = []

while True:
    # Capturing frame-by-frame
    ret, frame = video.read()

    # If the frame was not retrieved e.g.: at the end of the video, then we break the loop
    if not ret:
        break

    # Getting spatial dimensions of the frame as we do it only once from the very beginning
    # all other frames have the same dimension
    if w is None or h is None:
        # Slicing from tuple only first two elements
        h, w = frame.shape[:2]
    
    """
    End of: Reading frame in loop
    """

    """
    ==================     STEP4  ===================
    Start of: Getting blob from current frame
    """

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    """
    End of: Getting blob from current frame
    """

    """
    ==================     STEP4  ===================
    Start of: Implementing Forward pass
    """

    # Implementing forward pass with our blob and only through output layers
    # Calculating at the same time, needed time for forward pass
    network.setInput(blob)  # setting blob as input to the network
    output_from_network = network.forward(layers_names_output)


    # Increasing counters for frames and total time
    f += 1

    # Showing spent time for single current frame
    print('Processing frame number {0}'.format(f))
    
    """
    End of:Implementing Forward pass
    """
    
    """
    ==================     STEP5  ===================
    Start of: Getting bounding boxes
    """

    # Preparing lists for detected bounding boxes, obtained confidences and class's number
    boxes = []

    confidences = []

    current_labels = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]
            
            # # Every 'detected_objects' numpy array has first 4 numbers with
            # # bounding box coordinates and rest 80 with probabilities
            #  # for every class

            # Eliminating weak predictions with minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                # YOLO data format keeps coordinates for center of bounding box
                # and its current width and height
                # That is why we can just multiply them elementwise
                # to the width and height
                # of the original frame and in this way get coordinates for center
                # of bounding box, its width and height for original frame
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Now, from YOLO data format, we can get top left corner coordinates that are x_min and y_min
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                current_labels.append(class_current)
            	
        """	
        End of: Getting bounding boxes
        """

    """
    ==================     STEP6   ===================
    Start of: Non-maximum suppression
    """

    # Implementing non-maximum suppression of given bounding boxes
    # With this technique we exclude some of bounding boxes if their
    # corresponding confidences are low or there is another
    # bounding box for this region with higher confidence

    # It is needed to make sure that data type of the boxes is 'int'
    # and data type of the confidences is 'float'
    # https://github.com/opencv/opencv/issues/12789
    results = cv2.dnn.NMSBoxes(boxes, confidences,
                               probability_minimum, threshold)
    
    """
    End of: Non-maximum suppression
    """

    """
    ==================     STEP6   ===================
    Start of: Drawing bounding boxes and labels
    """

    # Checking if there is at least one detected object after non-maximum suppression
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Getting current bounding box coordinates, its width and height

            x_min, y_min, box_width, box_height = boxes[i]

            # Preparing colour for current bounding box and converting from numpy array to list
            colour_box_current = colours[current_labels[i]].tolist()


            label = str(labels[current_labels[i]])

            # Drawing bounding box on the original current frame
            if label in ['car', 'truck', 'bus', 'person']:
                cv2.rectangle(frame, (x_min, y_min), (x_min + box_width, y_min + box_height), colour_box_current, 2)


            #Calculating the pixel position of the center of the bounding box
            center = (x_min + box_width//2, y_min + box_height//2)

            frame_height = frame.shape[0]
            frame_width = frame.shape[1]

            #If the object in question is a vehicle
            if(label in ['car', 'truck', 'bus']):
                
                text_box_current = '{}: {:.4f}'.format(label, confidences[i])

                #Check if car crossed line or not by checking closest center in the old frame
                
                if(is_above_line_2D((0,int(y_threshold_1*frame_height)), (frame_width,int(y_threshold_2*frame_height)),center)):
                    dists = []
                    if(len(prev_results) > 0):
                        for j in prev_results.flatten():
                            if(label == labels[int(prev_labels[j])]):
                                prev_x_min, prev_y_min, prev_box_width, prev_box_height = prev_boxes[j]
                                prev_center = (prev_x_min + prev_box_width//2, prev_y_min + prev_box_height//2)
                                dists.append([prev_center[0],prev_center[1],euclid_2D(center,prev_center)])
                        if(len(dists) > 0):
                            dists = np.array(dists)
                            dists = dists[dists[:, 2].argsort()] 
                            if(not is_above_line_2D((0,int(y_threshold_1*frame_height)), (frame_width,int(y_threshold_2*frame_height)),(dists[0,0],dists[0,1]))):
                                total_cars_count += 1
                                if(dists[0,0] < parking_x_threshold * frame_width or dists[0,0] > (1-parking_x_threshold) * frame_width):
                                    parked_cars_count += 1


            #If the object in question is a pedestrian/person
            elif(label == 'person'):

                text_box_current = '{}: {:.4f}'.format(label, confidences[i])

                #Check if car crossed line or not by checking closest center in the old frame
                if(is_above_line_2D((0,int(pedestrian_y_threshold_1*frame_width)), (frame_width,int(pedestrian_y_threshold_2*frame.shape[0])),center)):
                    dists = []
                    if(len(prev_results) > 0):
                        for j in prev_results.flatten():
                            if(label == labels[int(prev_labels[j])]):
                                prev_x_min, prev_y_min, prev_box_width, prev_box_height = prev_boxes[j]
                                prev_center = (prev_x_min + prev_box_width//2, prev_y_min + prev_box_height//2)
                                dists.append([prev_center[0],prev_center[1],euclid_2D(center,prev_center)])

                        if(len(dists) > 0):
                            dists = np.array(dists)
                            dists = dists[dists[:, 2].argsort()] 
                            if(not is_above_line_2D((0,int(pedestrian_y_threshold_1*frame_width)), (frame_width,int(pedestrian_y_threshold_2*frame_width)),(dists[0,0],dists[0,1]))):
                                total_pedestrian_count += 1
            

    passed_car_count_str = '{}: {:d}'.format("Total Passed Cars",
                                            total_cars_count)
    passed_parked_car_count_str = '{}: {:d}'.format("Passed Parked Cars",
                                            parked_cars_count)
    passed_moving_car_count_str = '{}: {:d}'.format("Passed Moving Cars",
                                total_cars_count - parked_cars_count)
    passed_pedestrian_count_str = '{}: {:d}'.format("Passed Pedestrians",
                                            total_pedestrian_count)
    
    draw_label(frame, passed_car_count_str, (10, 20), (0,255,0))
    draw_label(frame, passed_parked_car_count_str, (10, 40), (0,255,0))
    draw_label(frame, passed_moving_car_count_str, (10, 60), (0,255,0))
    draw_label(frame, passed_pedestrian_count_str, (10, 80), (0,255,0))



    """
    End of:
    Drawing bounding boxes and labels
    """
    
    
    """
    ==================     STEP7   ===================
    Start of: Writing processed frame into the file
    """

    # Initializing writer
    # we do it only once from the very beginning when we get spatial dimensions of the frames
    if writer is None:
        # Constructing code of the codec to be used in the function VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter('result.mp4', fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)
    
    # Write processed current frame to the file
    writer.write(frame)
    
    prev_boxes = boxes
    prev_results = results
    prev_labels = current_labels

"""
End of: Writing processed frame into the file
"""

"""
End of: Writing processed frame into the file
"""


# Printing final results
print()
print('Total number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))


# Releasing video reader and writer
video.release()
writer.release()


[200 227 254]
Frame number 1 took 0.34231 seconds
Frame number 2 took 0.09232 seconds
Frame number 3 took 0.09160 seconds
Frame number 4 took 0.08507 seconds
Frame number 5 took 0.08805 seconds
Frame number 6 took 0.08978 seconds
Frame number 7 took 0.08926 seconds
Frame number 8 took 0.08577 seconds
Frame number 9 took 0.08214 seconds
Frame number 10 took 0.08537 seconds
Frame number 11 took 0.08341 seconds
Frame number 12 took 0.08524 seconds
Frame number 13 took 0.08420 seconds
Frame number 14 took 0.08533 seconds
Frame number 15 took 0.08427 seconds
Frame number 16 took 0.08172 seconds
Frame number 17 took 0.08070 seconds
Frame number 18 took 0.08502 seconds
Frame number 19 took 0.08590 seconds
Frame number 20 took 0.08502 seconds
Frame number 21 took 0.08317 seconds
Frame number 22 took 0.08546 seconds
Frame number 23 took 0.08469 seconds
Frame number 24 took 0.08480 seconds
Frame number 25 took 0.08258 seconds
Frame number 26 took 0.08605 seconds
Frame number 27 took 0.08666 seco

IndexError: list index out of range